In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.12067055702209473
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 19.046588361781318
200000 31.683157034184962
300000 30.431137159654686
400000 29.8953240976653
500000 30.1708633956772
600000 31.05171394376628
700000 30.591480956094586
800000 31.050074535694943
900000 30.783155214925277
1000000 29.239088376789912
1100000 29.315452071080454
1200000 12.74598977976948
1300000 28.96330126102345
1400000 31.769907005743114
1500000 29.1057189693673
1600000 29.119284707995096
1700000 29.067385007509884
1800000 30.72523907269687
1900000 28.966363491596844
2000000 28.95014343776883
2100000 29.208016330896015
2200000 29.472723501584838
2300000 29.537646954769283
2400000 7.690999338651589
2500000 31.2796373312156
2600000 36.98707192535977
2700000 35.83283160460702
2800000 35.592892822833306
2900000 36.22616212860101
3000000 35.27825

29500000 16.78189402263129
29600000 26.923219048957666
29700000 28.315697468834152
29800000 28.061343072187505
29900000 29.07379661420249
30000000 27.574685829608487
30100000 35.04925204108589
30200000 35.896613106883194
30300000 29.16615479319134
30400000 12.488021128782483
30500000 29.483256685608776
30600000 29.37609239955882
30700000 30.627767410742052
30800000 26.01190562099316
30900000 15.239596861523115
31000000 29.21156910969237
31100000 29.803652879261694
31200000 28.947067328460225
31300000 28.175162070261567
31400000 28.54313815906708
31500000 28.124836080430615
31600000 27.414222424150886
31700000 26.66447344992829
31800000 7.98367305753898
31900000 27.75931265932381
32000000 26.490045402092743
32100000 27.799262724958506
32200000 28.096676551565846
32300000 28.56448489404964
32400000 28.424330199317268
32500000 29.45369688259095
32600000 28.666439842902683
32700000 19.388704573540565
32800000 21.914568012228386
32900000 29.32197076633553
33000000 29.996663304929527
3310000

59200000 27.222081710352505
59300000 11.65534159182533
59400000 24.804032595829604
59500000 25.41794218307704
59600000 25.520797655537656
59700000 23.589834969019574
59800000 23.762733340814933
59900000 23.742308839927976
60000000 13.599214338026442
60100000 17.555911919427288
60200000 24.23408013941055
60300000 23.688177365803984
60400000 24.05906432387377
60500000 24.8857808352043
60600000 24.904140992237885
60700000 21.106859737513666
60800000 16.463973208350314
60900000 27.359498405727468
61000000 30.44300219665552
61100000 29.063506815422027
61200000 27.077579150530678
61300000 27.602463939836586
61400000 27.013924208434467
61500000 27.485276905401548
61600000 25.913201884145014
61700000 10.514443000674934
61800000 25.35759657750598
61900000 25.941603388610257
62000000 25.01926838749692
62100000 24.711905474613335
62200000 23.68105912493923
62300000 24.604675960249267
62400000 6.172532208778602
62500000 25.38118520605078
62600000 23.82170422321278
62700000 24.440312344931844
62800

88900000 21.993219387946482
89000000 22.616808677165775
89100000 22.75905657545513
89200000 24.01713127288194
89300000 22.38384827102965
89400000 23.824327345798643
89500000 24.389472960900086
89600000 23.77406742439107
89700000 24.630134578615525
89800000 23.42283528891549
89900000 23.32882764650956
90000000 22.1131602208362
90100000 12.225056773798263
90200000 17.822365199300496
90300000 21.156738919429433
90400000 22.333390434398634
90500000 21.64511767656557
90600000 9.51446100356053
90700000 19.549016980785456
90800000 21.795900048250463
90900000 21.272891275737607
91000000 21.567996907578543
91100000 10.651286947329028
91200000 20.146234446098916
91300000 22.83828004021005
91400000 22.670696354555382
91500000 23.064999093450965
91600000 22.924589048454486
91700000 23.718798224510806
91800000 23.954996178376096
91900000 23.540041931529146
92000000 23.308463700042918
92100000 24.110022442114936
92200000 23.37174550835154
92300000 22.66160823108835
92400000 21.974265941472378
925000

118000000 20.899161124011346
118100000 20.00672445413896
118200000 20.50317268902574
118300000 19.363906566355688
118400000 19.6924234513171
118500000 17.273082158791734
118600000 9.56961717997365
118700000 19.88186077182638
118800000 13.491711951177438
118900000 11.331522355357599
119000000 19.355236597564147
119100000 12.463060981586175
119200000 15.558236808097517
119300000 19.755374628249147
119400000 19.11062605765518
119500000 19.861003543845165
119600000 19.900547031379777
119700000 20.407130935233912
119800000 20.770935769272047
119900000 21.548099139572386
120000000 21.03924037958118
120100000 21.30571509456604
120200000 21.06335808671644
120300000 20.703935809400996
120400000 19.446812710461828
120500000 19.49342396551241
120600000 19.78955234089624
120700000 19.208465607190142
120800000 19.238538189292196
120900000 12.949515959435356
121000000 13.792375085097346
121100000 18.356281069986775
121200000 5.681849173975173
121300000 19.147690855537054
121400000 16.78689643596059


...

261300000 10.44456498244872
261400000 10.16383369504352
261500000 8.853374901592785
261600000 5.591729447872398
261700000 10.342260102383031
261800000 9.314234419397659
261900000 9.958216197278936
262000000 9.259810402605211
262100000 9.183183373815337
262200000 9.289169997183844
262300000 7.130585237095407
262400000 5.316610348248027
262500000 8.654999290482356
262600000 8.74166918891099
262700000 9.852589380058584
262800000 9.308342470022344
262900000 10.001453852780521
263000000 9.683908753200443
263100000 8.672149700228738
263200000 5.630457148041877
263300000 10.393451186992973
263400000 9.873404488510436
263500000 10.363776313105024
263600000 10.246202583250614
263700000 10.258303573554171
263800000 10.01499232789643
263900000 3.819660098537864
264000000 11.388681161927297
264100000 9.764141235545537
264200000 10.064573170169405
264300000 9.363933538916635
264400000 10.053265336015933
264500000 9.025437621398094
264600000 9.514128620910743
264700000 3.5314235451267124
264800000 8

290600000 1.8157491572470388
290700000 7.962642614742872
290800000 7.452451981484854
290900000 8.061939249025167
291000000 7.560748481812823
291100000 7.930118554989971
291200000 7.609425582181402
291300000 8.114950168755596
291400000 7.704181543406143
291500000 8.346152574539639
291600000 2.769009707874881
291700000 8.090062467246991
291800000 7.984209488275131
291900000 4.144945886480646
292000000 7.375562949436222
292100000 7.587196547994738
292200000 8.039830131084766
292300000 7.605573100915103
292400000 8.016138082911938
292500000 7.315626148342212
292600000 8.08889483628416
292700000 7.542337812043311
292800000 7.864934988309785
292900000 4.736104584852024
293000000 4.588327206469091
293100000 7.794505566490068
293200000 7.580128624557054
293300000 8.02978332425133
293400000 7.65926345317125
293500000 8.300794856090945
293600000 7.761695112733258
293700000 8.014987411485718
293800000 7.675485571055617
293900000 7.567709640422121
294000000 3.4921154583323175
294100000 7.873601485

319500000 2.3542488224626394
319600000 2.318946636950666
319700000 2.428794573106096
319800000 2.437437709415952
319900000 2.5667825426742197
320000000 2.6817164906252966
320100000 2.478000992297151
320200000 2.69278186677956
320300000 2.3925553968200237
320400000 2.284174194779568
320500000 2.446076746285045
320600000 2.315351595994425
320700000 2.0214426908112793
320800000 2.40448052835928
320900000 2.0831011923206684
321000000 1.6887531351681007
321100000 1.2570107323726853
321200000 1.2268048755908243
321300000 1.2433123963973707
321400000 1.4928626207817077
321500000 2.398196058618675
321600000 2.6937955570287873
321700000 2.181512643981717
321800000 1.960567110326304
321900000 2.3658533123308643
322000000 2.419290658285143
322100000 2.3020326168910787
322200000 2.3182166094098497
322300000 2.5964970905520914
322400000 2.466642061694942
322500000 2.4840375522263476
322600000 2.4899840883597273
322700000 2.3800576328209155
322800000 2.1941128393234552
322900000 2.093840160289369
32

348100000 1.4258182689468424
348200000 1.5018797720886747
348300000 1.613499010941083
348400000 1.3500591170075715
348500000 1.3617935096522813
348600000 1.3593231580560632
348700000 1.4200215269637149
348800000 1.1458217966499118
348900000 1.2456463634241919
349000000 1.4517439500974159
349100000 1.4010229730000827
349200000 0.9977510873850757
349300000 0.6561744806593537
349400000 0.7195644634890708
349500000 0.729964909085885
349600000 0.9081957394599657
349700000 1.091048297150529
349800000 1.4732398692458708
349900000 1.2131004653755129
350000000 1.3063385442955369
350100000 1.3662522170529334
350200000 1.2625747892911314
350300000 1.2871532421491325
350400000 1.3953479660419543
350500000 1.321497270180422
350600000 1.4685998997713983
350700000 1.3314237590308353
350800000 1.3575803594721616
350900000 1.2620526397466163
351000000 1.2477777500254432
351100000 1.2542391296961546
351200000 1.2712275681287262
351300000 1.234144947954976
351400000 1.2712629556394677
351500000 1.0263619

375900000 0.04600225720836719
376000000 0.03768557572599252
376100000 0.03923163299595184
376200000 0.04235203944746653
376300000 0.05691408166136742
376400000 0.04316224188073079
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.25998336170698805
376600000 0.28246263238950176
376700000 0.28524133870855506
376800000 0.2999377187257813
376900000 0.2635175356276075
377000000 0.20746911992105907
377100000 0.19699609456228284
377200000 0.19283198029506204
377300000 0.18871298258818256
377400000 0.19754975260872706
377500000 0.24250026626261473
377600000 0.202208994461347
377700000 0.238803490289659
377800000 0.20315602589618167
377900000 0.14961765000725522
378000000 0.1961509932746914
378100000 0.2250853006479323
378200000 0.19766241051277186
378300000 0.21914036008816953
378400000 0.18758545698304574
378500000 0.25331841351635453
378600000 0.22560250178813404
378700000 0.2031222120663782
378800000 0.18249870837124982
378900000 0.03990247383317153
379000000